In [9]:
class node:
    def __init__(self,x,y):
        self.x = x
        self.y = y

a_node = node(10,20)
b_node = node(20,10)

for _ in range(3):
    if (a_node.x,a_node.y) == (b_node.x,b_node.y):
        print("YES")
    else:
        print("NO")
        # b_node.x = a_node.x
        # b_node.y = a_node.y
        (a_node.x,a_node.y) = (b_node.x,b_node.y)

print(a_node.x,a_node.y,b_node.x,b_node.y)

NO
YES
YES
20 10 20 10
